In [128]:
from openai import OpenAI
import pandas as pd
import os
import json
import time


df = pd.read_excel('/Users/andreasbrogaard/Documents/dsp_project/EDC w. description.xlsx')

GPT_response = pd.DataFrame()



Empty DataFrame
Columns: []
Index: []


In [130]:
system_message = ("You are a parameter identifier you are to identify the parameters. Your output is to json and only json!. Do not make an introduction or summary of your response")

parameters_schema = {
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": 'Divide locations into categories such as downtown, suburban, rural.',
        },
        "Number of bathrooms": {
            "type": "string",
            "enum": ['1-1.5 bathrooms', '2-2.5 bathrooms', '3+ bathrooms'],
            "description": 'The stated number of bathrooms. If no number of bathrooms is provided, return "not_stated".',
        },
        "Condition of the house": {
            "type": "string",
            "enum": ["new construction", "renovated", "fixer-upper"],
            "description": 'Return the condition of the unit as mentioned".',
        },
        "amenities": {
            "type": "array",
            "Balcony": {
                "type": "Boolean",
            },
            "Garden": {
                "type": "Boolean",
            },
            "Parking": {
                "type": "Boolean",
            },
            "Swimming pool": {
                "type": "Boolean",
            },
            "Fireplace": {
                "type": "Boolean",
            },
            "Garage": {
                "type": "Boolean",
            },
            "Basement": {
                "type": "Boolean",
            },
            "Elevator": {
                "type": "Boolean",
            },
            "Air conditioning": {
                "type": "Boolean",
            },
            "Heating Type": {
                "type": "string",
            },
            "description": 'Return the amenities mentioned in the description.',
        },
        "view": {
            "type": "string",
            "enum": ["city", "sea", "no view", "not_stated"],
            "description": 'Return the view mentioned in the description.',
        },
    },
    #"required": ["location", "budget_level", "purpose"],
}




In [192]:
OPENAI_API_KEY = open('/Users/andreasbrogaard/Documents/dsp_project/.env','r').read()

client = OpenAI(
    api_key = OPENAI_API_KEY
)

#Problem i row 57
for i in range(248, df.shape[0]):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message + "use the following schema to create a response" + str(parameters_schema)},
            {"role": "user", "content": df['Description'].iloc[i]}
            ],
        temperature= 0.1
        )
    
    response = completion.to_dict()['choices'][0]['message']['content']

    json_response = json.loads(response)

    df_response = pd.json_normalize(json_response)

    print(df_response)

    full_response = pd.concat([df.iloc[i], df_response.transpose()], axis=0, ignore_index=True).transpose()

    GPT_response = pd.concat([GPT_response, full_response], axis = 0 ,ignore_index=True)

    time.sleep(10)
    

print(GPT_response)


   location Number of bathrooms Condition of the house     view  \
0  suburban     1-1.5 bathrooms             not_stated  no view   

   amenities.Balcony  amenities.Garden  amenities.Parking  \
0              False              True               True   

   amenities.Swimming pool  amenities.Fireplace  amenities.Garage  \
0                    False                False             False   

   amenities.Basement  amenities.Elevator  amenities.Air conditioning  \
0               False               False                       False   

  amenities.Heating Type  
0             not_stated  
   location Number of bathrooms Condition of the house     view  \
0  suburban     2-2.5 bathrooms       new construction  no view   

   amenities.Balcony  amenities.Garden  amenities.Parking  \
0              False             False               True   

   amenities.Swimming pool  amenities.Fireplace  amenities.Garage  \
0                    False                False             False   

   am

In [194]:
os.makedirs('0. Data/GPT flows', exist_ok=True)
GPT_response.to_pickle('0. Data/GPT flows/House_data_w_GPT_1-270.pkl')


In [195]:
#GPT_response.shape

#print(response)

df.shape

(271, 15)

In [183]:
#import pandas as pd

# Read the pickle file
#data = pd.read_pickle("0. Data/GPT flows/House_data_w_GPT_1-50.pkl")

# Print the data
# Save the DataFrame as an Excel file
GPT_response.to_excel('gpt_response.xlsx', index=False)
